In [ ]:
import numpy as np
import shap
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
#Link: https://github.com/slundberg/shap

In [ ]:
m = 1000
n = 2
X = pd.DataFrame(np.random.rand(m, n), columns=['x'+ str(i) for i in range(1, n+1)])
y = 3*X.iloc[:,0] + 6*X.iloc[:,1]

In [ ]:
Xy = X.copy()
Xy['target'] = y

In [ ]:
fig = px.scatter_3d(Xy, 'x1', 'x2', 'target')
fig.update_traces(marker_size=2)

In [ ]:
model = LinearRegression().fit(X, y)
y_pred = model.predict(X)
Xy['pred'] = y_pred

In [ ]:
plt.scatter(Xy['target'], Xy['pred'])
print(model.score(X, y))
print(model.coef_)

In [ ]:
num_data_rows = 1000
shap_explainer = shap.LinearExplainer(model, X[:num_data_rows],
                                      nsamples=100,
                                      feature_perturbation=None)
#Different model for Tree based model and NN and generic models
print('mean model prediction: ', model.predict(Xy[:num_data_rows][['x1', 'x2']].mean()[None,:])[0])
print('shap_explainer expected_value: ', shap_explainer.expected_value)

In [ ]:
X_vals = pd.DataFrame([[0, 0],
                      [1, 1],
                      [1, 0],
                      [0.5, 0.5]], columns=X.columns)
shap_values = shap_explainer.shap_values(X_vals)
i = 1
print(shap_values[i])
print(((X[:num_data_rows].mean() - X_vals.iloc[i])*model.coef_).tolist())
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values[i], features=X_vals.iloc[i].values,
               matplotlib=False, feature_names=X_vals.columns, out_names='target')

In [ ]:
shap.force_plot(shap_explainer.expected_value, shap_values, features=X_vals.values,
               matplotlib=False, feature_names=X_vals.columns, out_names='target')

In [ ]:
#Feature importance
shap_values = shap_explainer.shap_values(X)
shap.summary_plot(shap_values, X)
shap.summary_plot(shap_values, X, plot_type='violin')
shap.summary_plot(shap_values, X, plot_type='bar')

In [ ]:
#Partial Dependence Plots
shap.dependence_plot("x1", shap_values, X)
shap.dependence_plot("x2", shap_values, X)